In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import ast

# BEST MOVIES

In [2]:
oscars_csv_file = "Resources/oscar.csv"
oscars_df = pd.read_csv(oscars_csv_file)

In [3]:
best_movie = oscars_df.loc[(oscars_df['Award'] == 'Best Motion Picture') | (oscars_df['Award'] == 'Best Picture')]
best_movie.head()

,Year,Ceremony,Award,Winner,Name,Film,Film2
1752,1944,17,Best Motion Picture,NaN,Double Indemnity,Paramount,Paramount
1753,1944,17,Best Motion Picture,NaN,Gaslight,Metro-Goldwyn-Mayer,Metro-Goldwyn-Mayer
1754,1944,17,Best Motion Picture,1.0,Going My Way,Paramount,Paramount
1755,1944,17,Best Motion Picture,NaN,Since You Went Away,Selznick International Pictures,Selznick International Pictures
1756,1944,17,Best Motion Picture,NaN,Wilson,20th Century-Fox,20th Century-Fox


In [4]:
# Cleaning the data
best_movie = best_movie[['Year', 'Name', 'Film', 'Winner']]

best_movie = best_movie.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

best_movie.head()

,Year,Name,Film,Winner
1752,1944,Double Indemnity,Paramount,NaN
1753,1944,Gaslight,Metro-Goldwyn-Mayer,NaN
1754,1944,Going My Way,Paramount,1.0
1755,1944,Since You Went Away,Selznick International Pictures,NaN
1756,1944,Wilson,20th Century-Fox,NaN


In [5]:
best_movie = best_movie.rename(columns = {'Year' : 'release_year', 
                                          'Name':'title', 
                                          'Film':'production_companies',
                                          'Winner':'win'})

best_movie['win'] = np.where(best_movie['win']==1.0, 'Y', 'N')

In [6]:
best_movie.head()

,release_year,title,production_companies,win
1752,1944,Double Indemnity,Paramount,N
1753,1944,Gaslight,Metro-Goldwyn-Mayer,N
1754,1944,Going My Way,Paramount,Y
1755,1944,Since You Went Away,Selznick International Pictures,N
1756,1944,Wilson,20th Century-Fox,N


In [7]:
# Adding the imdb_id variables
movie_ids = pd.read_csv("Resources/movies_metadata.csv")
movie_ids = movie_ids[['imdb_id', 'title', 'release_date']]
movie_ids['release_year'] = movie_ids['release_date'].str[:4]
movie_ids.head() 

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,imdb_id,title,release_date,release_year
0,tt0114709,Toy Story,1995-10-30,1995
1,tt0113497,Jumanji,1995-12-15,1995
2,tt0113228,Grumpier Old Men,1995-12-22,1995
3,tt0114885,Waiting to Exhale,1995-12-22,1995
4,tt0113041,Father of the Bride Part II,1995-02-10,1995


## Adding the IDs


In [8]:
best_movie_ids = pd.merge(best_movie, 
                          movie_ids, on=['title', 'release_year'], how='left')
best_movie_ids.head()
best_movie = best_movie_ids

# RATINGS

In [9]:
# Loading the ratings data
ratings_csv_file = 'Resources/ratings.csv'
ratings_df = pd.read_csv(ratings_csv_file)
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [10]:
# Selectiong relevant variables
clean_ratings_df = ratings_df[['movieId', 'rating', 'userId']].copy()
clean_ratings_df.head()

,movieId,rating,userId
0,110,1.0,1
1,147,4.5,1
2,858,5.0,1
3,1221,5.0,1
4,1246,5.0,1


In [11]:
# Adding the movie id
links_csv_file = ('Resources/links.csv')
links_df = pd.read_csv(links_csv_file)
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [12]:
# Cleaning IDs
links_clean_df = links_df[["movieId","imdbId"]].copy()
links_clean_df.head()

,movieId,imdbId
0,1,114709
1,2,113497
2,3,113228
3,4,114885
4,5,113041


In [ ]:
# Reviews with movie ids
reviews_w_movie_ids = pd.merge(links_clean_df, clean_ratings_df, on ='movieId', how = 'left')
reviews_w_movie_ids.head()

In [ ]:
# Making the movie ids the same across all tables
reviews_w_movie_ids['imdbId'] = 'tt' + links_df['imdbId'].astype(str)
reviews_w_movie_ids = reviews_w_movie_ids.rename(columns = {'userId' : 'reviewer_id', 
                                                            'imdbId':'imdb_id'})
reviews_w_movie_ids = reviews_w_movie_ids[['imdb_id','rating', 'reviewer_id']]
reviews_w_movie_ids.head()
ratings = reviews_w_movie_ids

# ALL MOVIES

In [ ]:
movie_csv_file = "Resources/movies_metadata.csv"#dont forget to edit this
movie_df = pd.read_csv(movie_csv_file)
movie_df.head()

In [18]:
# Selecting relevant variables
clean_movie_df = movie_df[['budget','imdb_id', 'title', 'release_date','revenue', 'production_companies']]
clean_movie_df.head()

,budget,imdb_id,title,release_date,revenue,production_companies
0,30000000,tt0114709,Toy Story,1995-10-30,373554033.0,"[{'name': 'Pixar Animation Studios', 'id': 3}]"
1,65000000,tt0113497,Jumanji,1995-12-15,262797249.0,"[{'name': 'TriStar Pictures', 'id': 559}, {'na..."
2,0,tt0113228,Grumpier Old Men,1995-12-22,0.0,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'..."
3,16000000,tt0114885,Waiting to Exhale,1995-12-22,81452156.0,[{'name': 'Twentieth Century Fox Film Corporat...
4,0,tt0113041,Father of the Bride Part II,1995-02-10,76578911.0,"[{'name': 'Sandollar Productions', 'id': 5842}..."


In [22]:
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('name ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('id ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('1 ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('2 ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('3 ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('4 ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('5 ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('6 ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('7 ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('8 ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('9 ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('9 ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('0 ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace(': ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('""" ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace(', "" ?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('"?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('\[?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('\]?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('\{?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace('\}?' , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace("'''?" , '')
clean_movie_df['production_companies'] = clean_movie_df.production_companies.str.replace("', ?" , '')

clean_movie_df.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

,budget,imdb_id,title,release_date,revenue,production_companies
0,30000000,tt0114709,Toy Story,1995-10-30,373554033.0,Pixar Animation Studios
1,65000000,tt0113497,Jumanji,1995-12-15,262797249.0,"TriStar Pictures, Teitler Film, Interscope Com..."
2,0,tt0113228,Grumpier Old Men,1995-12-22,0.0,"Warner Bros., Lancaster Gate"
3,16000000,tt0114885,Waiting to Exhale,1995-12-22,81452156.0,Twentieth Century Fox Film Corporation
4,0,tt0113041,Father of the Bride Part II,1995-02-10,76578911.0,"Sandollar Productions, Touchstone Pictures"


In [23]:
all_movies = clean_movie_df
clean_movie_df.head()

,budget,imdb_id,title,release_date,revenue,production_companies
0,30000000,tt0114709,Toy Story,1995-10-30,373554033.0,Pixar Animation Studios
1,65000000,tt0113497,Jumanji,1995-12-15,262797249.0,"TriStar Pictures, Teitler Film, Interscope Com..."
2,0,tt0113228,Grumpier Old Men,1995-12-22,0.0,"Warner Bros., Lancaster Gate"
3,16000000,tt0114885,Waiting to Exhale,1995-12-22,81452156.0,Twentieth Century Fox Film Corporation
4,0,tt0113041,Father of the Bride Part II,1995-02-10,76578911.0,"Sandollar Productions, Touchstone Pictures"


# EXPORTING THE DATASETS
